# Moment Interaction Diagram

This example demonstrates how to generate moment interaction diagrams. We start by importing the necessary modules.

In [ ]:
import numpy as np
from sectionproperties.pre.library import concrete_rectangular_section

from concreteproperties import (
    Concrete,
    ConcreteLinear,
    ConcreteSection,
    RectangularStressBlock,
    SteelBar,
    SteelElasticPlastic,
)
from concreteproperties.results import MomentInteractionResults

## Assign Materials
The materials used in this example will be 32 MPa concrete and 500 MPa steel, specified in accordance with AS 3600:2018.

In [ ]:
concrete = Concrete(
    name="32 MPa Concrete",
    density=2.4e-6,
    stress_strain_profile=ConcreteLinear(elastic_modulus=30.1e3),
    ultimate_stress_strain_profile=RectangularStressBlock(
        compressive_strength=32,
        alpha=0.802,
        gamma=0.89,
        ultimate_strain=0.003,
    ),
    flexural_tensile_strength=3.4,
    colour="lightgrey",
)

steel = SteelBar(
    name="500 MPa Steel",
    density=7.85e-6,
    stress_strain_profile=SteelElasticPlastic(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
    ),
    colour="grey",
)

## Create Geometry and Concrete Section
The geometry used in this example is identical to that used in [Calculating Area Properties](area_properties.ipynb#Create-Geometry-and-Concrete-Section).

In [ ]:
geom = concrete_rectangular_section(
    b=400,
    d=600,
    dia_top=20,
    area_top=310,
    n_top=3,
    c_top=30,
    dia_bot=24,
    area_bot=450,
    n_bot=3,
    c_bot=30,
    conc_mat=concrete,
    steel_mat=steel,
)

conc_sec = ConcreteSection(geom)
conc_sec.plot_section()

## Moment Interaction Diagram
We perform a moment interaction analysis by calling the ``moment_interaction_diagram()`` method.

In [ ]:
mi_res = conc_sec.moment_interaction_diagram(progress_bar=False)

We can plot the moment interaction diagram by calling the ``plot_diagram()`` method.

In [ ]:
mi_res.plot_diagram()

What if we were interested in bending about the ``y`` axis? In this case the neutral axis angle would be $\theta = \pi / 2$. Let's generate a moment interaction diagram for this case. We must specify that the moments to plot are those about the ``y`` axis.

In [ ]:
mi_res = conc_sec.moment_interaction_diagram(theta=np.pi / 2, progress_bar=False)
mi_res.plot_diagram(moment="m_y")

## Plotting Multiple Diagrams
We can also plot multiple diagrams at once by using the ``MomentInteractionResults.plot_multiple_diagrams()`` class method. In this example we'll create four different reinforced concrete cross-sections with different reinforcement ratios.

In [ ]:
# create lists to hold results and labels
mi_results = []
labels = []

# create four different sections with increasing reinforcement
# and peform a moment interaction analysis
for idx in range(4):
    geom = concrete_rectangular_section(
        b=400,
        d=600,
        dia_top=16,
        area_top=200 * (idx + 1),
        n_top=6,
        c_top=66,
        dia_bot=16,
        area_bot=200 * (idx + 1),
        n_bot=6,
        c_bot=66,
        conc_mat=concrete,
        steel_mat=steel,
    )

    conc_sec = ConcreteSection(geom)
    mi_results.append(conc_sec.moment_interaction_diagram(progress_bar=False))
    labels.append(f"p = {0.01 * (idx + 1)}")

In [ ]:
# plot all the diagrams on one image
MomentInteractionResults.plot_multiple_diagrams(
    moment_interaction_results=mi_results, labels=labels, fmt="-"
)

## Positive & Negative Interaction Diagrams
We can combine positive and negative bending on one plot using the ``plot_multiple_diagrams()`` method. Here we genearte results for ``theta = 0`` and ``theta = np.pi`` and plot both diagrams.

In [ ]:
geom = concrete_rectangular_section(
    b=400,
    d=600,
    dia_top=20,
    area_top=310,
    n_top=3,
    c_top=30,
    dia_bot=24,
    area_bot=450,
    n_bot=3,
    c_bot=30,
    conc_mat=concrete,
    steel_mat=steel,
)
conc_sec = ConcreteSection(geom)

mi_res_pos = conc_sec.moment_interaction_diagram(progress_bar=False)
mi_res_neg = conc_sec.moment_interaction_diagram(theta=np.pi, progress_bar=False)

MomentInteractionResults.plot_multiple_diagrams(
    moment_interaction_results=[mi_res_pos, mi_res_neg],
    labels=["Positive", "Negative"],
    fmt="-",
)

In the above plot the bending capacity is higher for positive bending and not symmetric. This is because the bottom bars (3N24) provide more area than the top bars (3N20).

## Advanced Features
This section will describe some advanced features that can be used to control how the moment interaction diagram is generated and displayed.

The limits of the moment interaction diagram and any additional analysis points can be specified by defining control points. Control points are defined as a list of (``str``, ``float``) tuples. The following types of control points are used in ``concreteproperties``:

1. ``D`` - ratio of neutral axis depth to section depth, e.g. the (``"D"``, ``0.5``) control point will ensure a neutral axis at mid-depth is analysed.

2. ``d_n`` - neutral axis depth, e.g. the (``"d_n"``, ``200``) control point will ensure a neutral axis at 200 is analysed.

3. ``fy`` - yield ratio of the most extreme tensile bar, e.g. the (``"fy"``, ``0.5``) control point will ensure a neutral axis corresponding to a strain of half the yield strain in the extreme tensile bar is analysed. The (``"fy"``, ``1.0``) control point is the balanced point.

4. ``N`` - axial force, e.g. (``"N"``, ``3000e3``) ensures a neutral axis depth corresponding to a net axial force of 3000e3 is analysed.

5. ``kappa_0`` - zero curvature compression, this must be at start of the list as it results in the largest compression force. Note that the second value in the tuple is not used and can be anything.<br>

The following parameters can be used to modify the moment interaction diagram:

1. ``limits`` - List of control points that define the start and end of the interaction diagram. List length must equal two. The default limits range from concrete decompression strain ``("D", 1.0)`` to zero curvature tension ``("d_n", 1e-6)``.

2. ``control_points`` - adding control points to the moment interaction diagram allows the user to ensure certain points are captured on the moment interaction diagram.  The default argument for ``control_points`` is ``[("kappa0", 0.0), ("fy", 1.0), ("N", 0.0)]`` which produces control points at the pure compression point, the balanced point and the pure bending point. 
    

3. ``labels`` - A list of labels to apply to the ``limits`` and ``control_points`` for plotting purposes. The first two values in ``labels`` apply labels to the ``limits``, the remaining values apply labels to the ``control_points``.

4. ``n_points`` - The number of points to compute including and between the ``limits`` of the moment interaction diagram. Generates equally spaced neutral axis depths between the ``limits``.

5. ``n_spacing`` - If provided, overrides ``n_points`` and generates the moment interaction diagram using ``n_spacing`` equally spaced axial loads.

6. ``max_comp`` - Limits the maximum compressive force in the moment interaction diagram by cutting off the top of the diagram.

The below example will use these features, creating additional control points at:

- Decompression point for the concrete ``("D", 1.0)`` - label ``C``
- Decompression point for the steel ``("fy", 0.0)`` - label ``D``
- 50% yield strain in the steel ``("fy", 0.5)`` - label ``E``
- 100% yield strain in the steel ``("fy", 1.0)`` - label ``F``
- Neutral axis depth of 200 mm ``("d_n", 200.0)`` - label ``G``

A maximum compressive force of 6800 kN is provided. Label ``A`` is placed at zero bending moment and label ``B`` is placed at the intersection with the moment interaction diagram.

The limits of the moment interaction diagram are taken from:

- Pure compression ``("kappa0", 0)`` - note label ``NA`` is removed by ``max_comp``
- Pure tension ``("d_n", 1e-6)`` - label ``I``

The interaction diagram is generated with constant axial force spacing using 36 points.

In [ ]:
mi_res = conc_sec.moment_interaction_diagram(
    limits=[
        ("kappa0", 0.0),
        ("d_n", 1e-6),
    ],
    control_points=[
        ("D", 1.0),
        ("fy", 0.0),
        ("fy", 0.5),
        ("fy", 1.0),
        ("d_n", 200.0),
        ("N", 0.0),
    ],
    labels=["NA", "I", "C", "D", "E", "F", "G", "H"],
    n_spacing=36,
    max_comp=6.8e6,
    max_comp_labels=["A", "B"],
    progress_bar=False,
)

To display the labels set ``labels=True``. The argument ``label_offset`` attempts to offset the labels from the diagram in a pretty way.

In [ ]:
import matplotlib.pyplot as plt

ax = mi_res.plot_diagram(fmt="-kx", labels=True, label_offset=True, render=False)

# reset axis limits to ensure labels are within plot
ax.set_xlim(-20, 850)
ax.set_ylim(-3000, 9000)
plt.show()